## Quick start

For a brief introduction to `Pyatoa`, let's generate the waveform image shown on the landing page. The code snippet below combines, formats and measures pre-collected waveform data using default values. 

In [3]:
from obspy import read
from pyatoa import Config, Manager, logger

# Ensure that we get the maximum level of under-the-hood output
logger.setLevel("DEBUG")

# This simple example will read in ObsPy's example data
st_obs = read()
st_syn = st_obs.copy()

# Pyatoa can be initiated and run in a few lines
cfg = Config(min_period=10, max_period=30)
mgmt = Manager(config=cfg, st_obs=st_obs, st_syn=st_syn)
mgmt.flow()
mgmt.plot(choice="wav", show=True)

[2022-01-28 16:37:34] - pyatoa - DEBUG: Component list set to E/N/Z
[2022-01-28 16:37:34] - pyatoa - INFO: standardizing streams
[2022-01-28 16:37:34] - pyatoa - DEBUG: start and endtimes already match to 0.001


ManagerError: cannot preprocess, no inventory